In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 18:01:24.696360 139652427728704 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 18:01:24.703412 139652427728704 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 18:01:24.708506 139652427728704 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 18:01:24.714673 139652427728704 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 18:01:24.720474 139652427728704 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 18:01:24.728435 139652427728704 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 18:01:24.769569 139652427728704 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 18:01:24.843519 139652427728704 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9283 train_acc= 0.3286 val_loss= 1.9319 val_acc= 0.2833 time= 1.0792
Epoch: 0002 train_loss= 1.9100 train_acc= 0.3857 val_loss= 1.9168 val_acc= 0.3400 time= 0.0171
Epoch: 0003 train_loss= 1.8907 train_acc= 0.4143 val_loss= 1.9011 val_acc= 0.3800 time= 0.0167
Epoch: 0004 train_loss= 1.8702 train_acc= 0.4643 val_loss= 1.8844 val_acc= 0.4200 time= 0.0163
Epoch: 0005 train_loss= 1.8497 train_acc= 0.4786 val_loss= 1.8677 val_acc= 0.4433 time= 0.0173
Epoch: 0006 train_loss= 1.8292 train_acc= 0.4929 val_loss= 1.8509 val_acc= 0.4667 time= 0.0169
Epoch: 0007 train_loss= 1.8087 train_acc= 0.4786 val_loss= 1.8344 val_acc= 0.4633 time= 0.0165
Epoch: 0008 train_loss= 1.7891 train_acc= 0.4714 val_loss= 1.8187 val_acc= 0.4733 time= 0.0173
Epoch: 0009 train_loss= 1.7704 train_acc= 0.4643 val_loss= 1.8039 val_acc= 0.4633 time= 0.0165
Epoch: 0010 train_loss= 1.7528 train_acc= 0.4571 val_loss= 1.7902 val_acc= 0.4467 time= 0.0166
Epoch: 0011 train_loss= 1.7365 train_acc= 0.4500 v

Epoch: 0096 train_loss= 0.7298 train_acc= 0.8857 val_loss= 1.0197 val_acc= 0.7500 time= 0.0182
Epoch: 0097 train_loss= 0.7234 train_acc= 0.8857 val_loss= 1.0142 val_acc= 0.7533 time= 0.0188
Epoch: 0098 train_loss= 0.7172 train_acc= 0.8857 val_loss= 1.0093 val_acc= 0.7533 time= 0.0171
Epoch: 0099 train_loss= 0.7110 train_acc= 0.8929 val_loss= 1.0045 val_acc= 0.7567 time= 0.0164
Epoch: 0100 train_loss= 0.7046 train_acc= 0.8929 val_loss= 0.9993 val_acc= 0.7633 time= 0.0170
Epoch: 0101 train_loss= 0.6986 train_acc= 0.9000 val_loss= 0.9940 val_acc= 0.7600 time= 0.0167
Epoch: 0102 train_loss= 0.6930 train_acc= 0.9000 val_loss= 0.9892 val_acc= 0.7533 time= 0.0165
Epoch: 0103 train_loss= 0.6874 train_acc= 0.9071 val_loss= 0.9846 val_acc= 0.7600 time= 0.0166
Epoch: 0104 train_loss= 0.6816 train_acc= 0.9071 val_loss= 0.9795 val_acc= 0.7600 time= 0.0166
Epoch: 0105 train_loss= 0.6757 train_acc= 0.9071 val_loss= 0.9744 val_acc= 0.7667 time= 0.0165
Epoch: 0106 train_loss= 0.6698 train_acc= 0.9071 v

Epoch: 0193 train_loss= 0.3924 train_acc= 0.9786 val_loss= 0.7606 val_acc= 0.8233 time= 0.0170
Epoch: 0194 train_loss= 0.3914 train_acc= 0.9786 val_loss= 0.7615 val_acc= 0.8200 time= 0.0167
Epoch: 0195 train_loss= 0.3899 train_acc= 0.9714 val_loss= 0.7612 val_acc= 0.8200 time= 0.0168
Epoch: 0196 train_loss= 0.3877 train_acc= 0.9714 val_loss= 0.7584 val_acc= 0.8200 time= 0.0172
Epoch: 0197 train_loss= 0.3853 train_acc= 0.9714 val_loss= 0.7546 val_acc= 0.8200 time= 0.0166
Epoch: 0198 train_loss= 0.3826 train_acc= 0.9714 val_loss= 0.7505 val_acc= 0.8200 time= 0.0188
Epoch: 0199 train_loss= 0.3804 train_acc= 0.9714 val_loss= 0.7473 val_acc= 0.8233 time= 0.0170
Epoch: 0200 train_loss= 0.3785 train_acc= 0.9714 val_loss= 0.7460 val_acc= 0.8200 time= 0.0180


In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8094
accuracy = 0.8130


# Gaussian Graph Convolution

In [15]:
B1 = 5e-4
B2 = 5e-4

In [16]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
#     dropout=0.5
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 18:01:29.377156 139652427728704 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:28: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 18:01:29.378515 139652427728704 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update

In [17]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)

W1110 18:01:29.415266 139652427728704 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:28: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [19]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9387 train_acc= 0.3214 val_loss= 1.9422 val_acc= 0.1933 time= 0.4881
Epoch: 0002 train_loss= 1.9320 train_acc= 0.3714 val_loss= 1.9372 val_acc= 0.2633 time= 0.0172
Epoch: 0003 train_loss= 1.9226 train_acc= 0.3500 val_loss= 1.9302 val_acc= 0.2833 time= 0.0174
Epoch: 0004 train_loss= 1.9115 train_acc= 0.3643 val_loss= 1.9216 val_acc= 0.3067 time= 0.0167
Epoch: 0005 train_loss= 1.9008 train_acc= 0.3786 val_loss= 1.9141 val_acc= 0.2867 time= 0.0168
Epoch: 0006 train_loss= 1.8883 train_acc= 0.4214 val_loss= 1.9055 val_acc= 0.3267 time= 0.0165
Epoch: 0007 train_loss= 1.8775 train_acc= 0.4357 val_loss= 1.8967 val_acc= 0.3433 time= 0.0164
Epoch: 0008 train_loss= 1.8626 train_acc= 0.4500 val_loss= 1.8863 val_acc= 0.3800 time= 0.0177
Epoch: 0009 train_loss= 1.8488 train_acc= 0.4429 val_loss= 1.8760 val_acc= 0.3833 time= 0.0172
Epoch: 0010 train_loss= 1.8367 train_acc= 0.4500 val_loss= 1.8663 val_acc= 0.3967 time= 0.0167
Epoch: 0011 train_loss= 1.8194 train_acc= 0.4643 v

Epoch: 0097 train_loss= 0.8471 train_acc= 0.8786 val_loss= 1.0999 val_acc= 0.7200 time= 0.0176
Epoch: 0098 train_loss= 0.8397 train_acc= 0.8929 val_loss= 1.0946 val_acc= 0.7200 time= 0.0230
Epoch: 0099 train_loss= 0.8329 train_acc= 0.8929 val_loss= 1.0889 val_acc= 0.7267 time= 0.0171
Epoch: 0100 train_loss= 0.8259 train_acc= 0.8929 val_loss= 1.0836 val_acc= 0.7333 time= 0.0186
Epoch: 0101 train_loss= 0.8196 train_acc= 0.8929 val_loss= 1.0783 val_acc= 0.7400 time= 0.0173
Epoch: 0102 train_loss= 0.8122 train_acc= 0.9071 val_loss= 1.0729 val_acc= 0.7400 time= 0.0167
Epoch: 0103 train_loss= 0.8059 train_acc= 0.9071 val_loss= 1.0677 val_acc= 0.7400 time= 0.0173
Epoch: 0104 train_loss= 0.7995 train_acc= 0.9071 val_loss= 1.0624 val_acc= 0.7400 time= 0.0177
Epoch: 0105 train_loss= 0.7927 train_acc= 0.9071 val_loss= 1.0574 val_acc= 0.7400 time= 0.0180
Epoch: 0106 train_loss= 0.7860 train_acc= 0.9071 val_loss= 1.0519 val_acc= 0.7400 time= 0.0184
Epoch: 0107 train_loss= 0.7795 train_acc= 0.9071 v

Epoch: 0192 train_loss= 0.4438 train_acc= 0.9714 val_loss= 0.7964 val_acc= 0.8300 time= 0.0172
Epoch: 0193 train_loss= 0.4414 train_acc= 0.9714 val_loss= 0.7940 val_acc= 0.8333 time= 0.0173
Epoch: 0194 train_loss= 0.4392 train_acc= 0.9714 val_loss= 0.7923 val_acc= 0.8333 time= 0.0172
Epoch: 0195 train_loss= 0.4368 train_acc= 0.9714 val_loss= 0.7900 val_acc= 0.8300 time= 0.0174
Epoch: 0196 train_loss= 0.4347 train_acc= 0.9714 val_loss= 0.7876 val_acc= 0.8333 time= 0.0174
Epoch: 0197 train_loss= 0.4320 train_acc= 0.9714 val_loss= 0.7845 val_acc= 0.8300 time= 0.0176
Epoch: 0198 train_loss= 0.4301 train_acc= 0.9714 val_loss= 0.7827 val_acc= 0.8300 time= 0.0168
Epoch: 0199 train_loss= 0.4278 train_acc= 0.9714 val_loss= 0.7798 val_acc= 0.8300 time= 0.0167
Epoch: 0200 train_loss= 0.4259 train_acc= 0.9714 val_loss= 0.7778 val_acc= 0.8300 time= 0.0175


In [20]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8601
accuracy = 0.8280


# Attacks